In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.chains import ConversationChain
from typing import Optional, List, Any
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import requests


In [2]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
def load_index_and_meta(index_path, meta_path):
    index = faiss.read_index(index_path)
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    return index, meta

def search_faiss(query, index, meta, top_k=2):
    query_vec = embedder.encode([query])
    query_vec = np.array(query_vec).astype("float32")
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        if idx < len(meta):
            results.append(meta[idx])
    return results
class ollama(LLM):
    model_name: str = "mistral"

    @property
    def _llm_type(self) -> str:
        return "ollama"
    
    def _call(self,prompt: str, stop: Optional[List[str]] = None, **kwargs: Any):
        response = requests.post(
        "http://127.0.0.1:11434/api/generate",
        json={"model": self.model_name, "prompt": prompt, "stream": False}
    )
        return response.json()["response"]

In [4]:
ipc_index, ipc_meta = load_index_and_meta("ipc.index", "ipc.json")
judg_index, judg_meta = load_index_and_meta("judgments.index", "judgments.json")

In [ ]:
custom=PromptTemplate(input_variables=["history","input"],template=""" You are an legal Chatbot assistant specializing in India Penal Code.
                      
                      Previous conversations:{history}
                      
                      Current question and also judgements :{input}
                      
                    provide an suitable reply for the current question by utilizing previous conversations and keep it around 20 to 30 words .
                      Utilize similar case judgement and compare it with the user case summary and then reply but do not repeat about judgements in every replies.
                      Only use the given text as knowledge, Do Not Retrieve your own knowledge
                      Only Reply using the given IPC sections, STICK TO ONLY THIS TEXT AND PREVIOUS CONVERSATION FOR KNOWLEDGE.
                      Consider youself as an legal advisor and you are the professional.""")

In [6]:
llm=ollama()
memory=ConversationBufferMemory()
conversion = ConversationChain(llm=llm,memory=memory,prompt=custom,verbose=False)

C:\Users\nihca\AppData\Local\Temp\ipykernel_4152\3123097401.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
C:\Users\nihca\AppData\Local\Temp\ipykernel_4152\3123097401.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversion = ConversationChain(llm=llm,memory=memory,prompt=custom,verbose=False)


In [8]:
print("⚖️ IPC + Judgement Chatbot (type 'exit' to quit)\n")

user_input = input("You: ").strip()


ipc_results = search_faiss(user_input, ipc_index, ipc_meta)
judg_results = search_faiss(user_input, judg_index, judg_meta)

        # Step 2: Build context
context = "Relevant IPC Sections:\n"
for sec in ipc_results:
    context += f"- Section {sec['section_no']}: {sec['Description']} (Punishment: {sec['punishment_raw']})\n"

prompt = f"User Query: {user_input}\n\n{context}\n\nBased on the above IPC sections, Name the sections relating to the user query and also list possible punishments."
Reply_1=llm._call(prompt)

context += "\nRelevant Judgements:\n"
for case in judg_results:
    context += f"""- Case facts: {case['case_facts']}
Issues: {case['issues']}
Arguments: {case['arguments']}
Decision: {case['decision']}
"""



print(f"\nBot: {Reply_1}\n")

    


⚖️ IPC + Judgement Chatbot (type 'exit' to quit)


Bot:  Based on the user's query, the relevant Indian Penal Code (IPC) sections that could potentially apply are:

1. Section 324 - Voluntarily causing hurt by dangerous weapons or means: This section might apply because the user admitted to killing their friend, which could be interpreted as causing hurt voluntarily. The punishment for this section includes imprisonment for a term extending up to three years, fine, or both.

2. Section 302 - Murder: If the act of causing harm resulted in death, then Section 302 might also apply. However, it's important to note that intent is a crucial factor in determining whether an act falls under this section. In this case, if the user intentionally caused their friend's death, they could be charged with murder, which carries a maximum punishment of death or life imprisonment.

3. Section 304- Part I - Causing death by rash or negligent act: If the harm was caused without the intent to kill but reck

In [ ]:
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    user_input+=f"\n\n similar case judgements : {context}"

    answer = conversion.predict(input=user_input)

    print(f"\nBot: {answer}\n")


Bot:  Based on the provided judgement, intentional harm resulting in death can be punishable under IPC Section 302 or 324. Circumstantial evidence can support a conviction if it forms a clear and unambiguous chain proving your involvement. Be aware that any intentional harm could lead to legal consequences.

